In [0]:
#Graph Genration and Dijkstria
#import geopandas as gpd
import pandas as pd
import re
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
import numpy as np

In [0]:
result = pd.read_csv('boston_safe_lat&long_filtered.csv')


In [0]:
#take 10%
sample = result.sample(frac=0.1, replace=False, random_state=1).reset_index().drop(['index'], axis = 1)

In [32]:
sample.head()

,Unnamed: 0,lat,long
0,8680,42.371892,-71.049632
1,6585,42.383380,-71.029959
2,4,42.314572,-71.130608
3,11730,42.359487,-71.167483
4,13455,42.259846,-71.103666


In [33]:
sample.shape

(1958, 3)

In [0]:
import random

In [0]:
#Final Dijkstria

from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
  return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path



In [36]:
#sample numbers to see hoe dijkstria works
graph = Graph([
    ("a", "b", 7),  ("a", "c", 9),  ("a", "f", 14), ("b", "c", 10),
    ("b", "d", 15), ("c", "d", 11), ("c", "f", 2),  ("d", "e", 6),
    ("e", "f", 9)])

print(graph.dijkstra("a", "e"))

deque(['a', 'c', 'd', 'e'])


In [37]:
#GRAPH GENERATION
from collections import Counter 

graph_list = []
start = sample.iloc[[random.choice(sample.index)]]
others = sample.drop(start.index[0])

while len(graph_list) < 100:

  dist_start_others = dict()
  for i in range(others.shape[0]):
      dist_start_others[str(others.index[i])] = haversine_np(start.iloc[0][0], start.iloc[0][1], others.iloc[i][0], others.iloc[i][1])
      layer_1 = dict()
      k = Counter(dist_start_others)
      high = k.most_common(2)
      for k in range(len(high)):
        layer_1[high[k][0]] = high[k][1]

  graph_list.append(tuple((str(start.index[0]), str(high[0][0]),  high[0][1] )))
  graph_list.append(tuple((str(start.index[0]), str(high[1][0]),  high[1][1] )))

  #Layer 11
  to_drop = []
  for j in range(len(list(layer_1.keys()))):
    to_drop.append(int(list(layer_1.keys())[j]))
  others_11 = others.drop(to_drop)
  node_11 = sample.iloc[[high[0][0]]]
  dist_11_2 = dict()
  for i in range(others_11.shape[0]):
      dist_11_2[str(others_11.index[i])] = haversine_np(node_11.iloc[0][0], node_11.iloc[0][1], others_11.iloc[i][0], others_11.iloc[i][1])
      layer_2u = dict()
      k = Counter(dist_11_2)
      high_11 = k.most_common(2)
      for k in range(len(high_11)):
        dist_11_2[high_11[k][0]] = high_11[k][1]

  graph_list.append(tuple((str(node_11.index[0]), str(high_11[0][0]),  high_11[0][1] )))
  graph_list.append(tuple((str(node_11.index[0]), str(high_11[1][0]),  high_11[1][1] )))


  #Layer 12
  #to_drop = []
  #for j in range(len(list(high_11.keys()))):
  #  to_drop.append(int(list(high_11_2.keys())[j]))
  #others_12 = others_11.drop(to_drop)
  others_12 = others_11
  node_12 = sample.iloc[[high[1][0]]]
  dist_12_2 = dict()
  for i in range(others_12.shape[0]):
      dist_12_2[str(others_12.index[i])] = haversine_np(node_12.iloc[0][0], node_12.iloc[0][1], others_12.iloc[i][0], others_12.iloc[i][1])
      layer_2l = dict()
      k = Counter(dist_12_2)
      high_12 = k.most_common(2)
      for k in range(len(high_12)):
        dist_12_2[high_12[k][0]] = high_12[k][1]

  graph_list.append(tuple((str(node_12.index[0]), str(high_12[0][0]),  high_12[0][1] )))
  graph_list.append(tuple((str(node_12.index[0]), str(high_12[1][0]),  high_12[1][1] )))


  others2 = others_11.drop(list(np.unique([int(high_11[0][0]), int(high_11[1][0]) , int(high_12[0][0]) , int(high_12[1][0]) ])))

  dist2 = dict()
  for i in range(others2.shape[0]):
    dist2[str(others2.index[i])] = haversine_np(sample.iloc[int(high_11[0][0])][0], sample.iloc[int(high_11[0][0])][1], others2.iloc[i][0], others2.iloc[i][1]) + haversine_np(sample.iloc[int(high_11[1][0])][0], sample.iloc[int(high_11[1][0])][1], others2.iloc[i][0], others2.iloc[i][1]) +  haversine_np(sample.iloc[int(high_12[0][0])][0], sample.iloc[int(high_12[0][0])][1], others2.iloc[i][0], others2.iloc[i][1]) +  haversine_np(sample.iloc[int(high_12[1][0])][0], sample.iloc[int(high_12[1][0])][1], others2.iloc[i][0], others2.iloc[i][1])

  node3 = min(dist2, key=dist2.get)
  dist3 = min(list(dist2.values()))
  graph_list.append(tuple((high_11[0][0], node3, haversine_np(sample.iloc[int(high_11[0][0])][0], sample.iloc[int(high_11[0][0])][1], others2.loc[int(node3)][0], others2.loc[int(node3)][1])  )))
  graph_list.append(tuple((high_11[1][0], node3, haversine_np(sample.iloc[int(high_11[1][0])][0], sample.iloc[int(high_11[1][0])][1], others2.loc[int(node3)][0], others2.loc[int(node3)][1])  )))
  graph_list.append(tuple((high_12[0][0], node3, haversine_np(sample.iloc[int(high_12[0][0])][0], sample.iloc[int(high_12[0][0])][1], others2.loc[int(node3)][0], others2.loc[int(node3)][1])  )))
  graph_list.append(tuple((high_12[1][0], node3, haversine_np(sample.iloc[int(high_12[1][0])][0], sample.iloc[int(high_12[1][0])][1], others2.loc[int(node3)][0], others2.loc[int(node3)][1])  )))

  others3 = others2.drop(int(node3))

  start = sample.iloc[[int(node3)]]
  others = others3

  print(len(graph_list))


10
20
30
40
50
60
70
80
90
100


In [38]:
graph_list

[('1791', '1496', 6585.711420517396),
 ('1791', '258', 6585.268385270796),
 ('1496', '1696', 6591.950005922558),
 ('1496', '1926', 6591.617280096686),
 ('258', '1926', 6591.174244961758),
 ('258', '1696', 6590.181445105518),
 ('1696', '1741', 0.5513356393225739),
 ('1926', '1741', 51.09492219554058),
 ('1926', '1741', 51.09492219554058),
 ('1696', '1741', 0.5513356393225739),
 ('1741', '630', 6590.820370643499),
 ('1741', '1328', 6588.744433442027),
 ('630', '680', 6589.187476700271),
 ('630', '1343', 6588.40527403693),
 ('1328', '520', 6588.490296250439),
 ('1328', '1243', 6587.826248690744),
 ('680', '1908', 102.1752142314697),
 ('1343', '1908', 3.1732955146628985),
 ('520', '1908', 102.1779372720162),
 ('1243', '1908', 51.133567362377676),
 ('1908', '1383', 6584.646478258368),
 ('1908', '1540', 6583.364635287966),
 ('1383', '1359', 6590.448106155919),
 ('1383', '337', 6590.43012881828),
 ('1540', '344', 6584.34202638325),
 ('1540', '1359', 6583.87127280092),
 ('1359', '760', 0.54949

In [0]:
gr3 = Graph(graph_list)

In [41]:
print(gr3.dijkstra("1791", "1788"))

deque(['1791', '258', '1696', '1741', '630', '1343', '1908', '1540', '1359', '760', '1114', '1686', '227', '644', '23', '1397', '104', '914', '214', '294', '1521', '1606', '1083', '1404', '329', '764', '190', '1053', '873', '1437', '1788'])
